In [1]:
import os
import sys
os.chdir("/Users/DanPfeiffer/Documents/code/data-collector/src")
sys.path.append(os.getcwd())
import pandas as pd
import importlib
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### TRAINING

In [2]:
training_df_raw = pd.read_csv('../data/transformed/training.csv')

/var/folders/h1/1n1src7x64q_jwgsr43hmkxm0000gq/T/ipykernel_20599/1458285592.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  training_df_raw = pd.read_csv('../data/transformed/training.csv')


In [3]:
training_df = training_df_raw.copy()

In [4]:
training_df["rest_differential"] = training_df["home_days_rest"] - training_df["away_days_rest"]

In [5]:
train_features = [
    'game_datetime',
    'home_offense_special_teams_net_punt_ypa',
    'offense_special_teams_net_punt_ypa_all_matchup_differential',
    'offense_special_teams_gross_punt_ypa_matchup_differential',
    'offense_special_teams_fg_made_pcnt_matchup_differential',
    'road_defense_passing_attempts',
    'road_defense_passing_completion_pcnt',
    'road_defense_passing_ypa',
    'road_defense_passing_ypc',
    'road_defense_passing_1st_downs',
    'road_defense_passing_td_pcnt',
    'road_defense_passing_pass_rtg',
    'road_defense_passing_sack_pcnt',
    'road_defense_passing_sacks',
    'road_defense_passing_1st_down_pcnt',
    'road_defense_rushing_attempts',
    'road_defense_rushing_ypg',
    'road_defense_rushing_ypa',
    'road_defense_rushing_td_pcnt',
    'road_defense_rushing_1st_down_pcnt',
    'road_defense_scoring_yards_per_point',
    'road_defense_scoring_rz_attempts',
    'road_defense_scoring_rz_td_pcnt',
    'road_def_turnovers_fumbles',
    'road_def_turnovers_takeaways',
    'road_def_turnovers_int_pcnt',
    'road_defense_total_plays',
    'road_defense_total_yards_per_play',
    'road_defense_total_3rd_downs',
    'road_defense_total_4th_downs',
    'road_defense_total_seconds_per_play',
    'road_defense_total_1st_downs_per_play',
    'road_defense_total_3rd_down_conversion_pcnt',
    'road_defense_total_4th_down_conversion_pcnt',
    'road_defense_total_punts_per_play',
    'road_defense_total_punts_per_score',
    'road_defense_total_solo_tackles',
    'road_defense_total_assisted_tackles',
    'home_off_turnovers_fumbles',
    'home_off_turnovers_int_pcnt',
    'off_turnovers_fumbles_matchup_differential',
    'off_turnovers_int_pcnt_matchup_differential',
    'home_offense_passing_completion_pcnt',
    'home_offense_passing_ypa',
    'home_offense_passing_ypc',
    'home_offense_passing_td_pcnt',
    'home_offense_passing_sack_pcnt',
    'home_offense_passing_1st_down_pcnt',
    'home_offense_passing_pass_rtg',
    'offense_passing_completion_pcnt_matchup_differential',
    'offense_passing_ypa_matchup_differential',
    'offense_passing_ypc_matchup_differential',
    'offense_passing_sack_pcnt_matchup_differential',
    'offense_passing_1st_downs_matchup_differential',
    'offense_passing_1st_down_pcnt_matchup_differential',
    'offense_passing_pass_rtg_matchup_differential',
    'home_offense_rushing_ypa',
    'home_offense_rushing_td_pcnt',
    'offense_rushing_ypa_matchup_differential',
    'offense_rushing_td_pcnt_matchup_differential',
    'home_offense_scoring_points_per_game',
    'home_offense_scoring_points_per_game_delta',
    'home_offense_scoring_yards_per_point',
    'home_offense_scoring_yards_per_point_delta',
    'home_offense_scoring_points_per_play',
    'home_offense_scoring_rz_td_pcnt',
    'offense_scoring_points_per_game_matchup_differential',
    'offense_scoring_points_per_game_delta_matchup_differential',
    'offense_scoring_yards_per_point_matchup_differential',
    'offense_scoring_yards_per_point_delta_matchup_differential',
    'offense_scoring_points_per_play_matchup_differential',
    'home_offense_total_yards',
    'home_offense_total_plays',
    'home_offense_total_yards_per_play',
    'home_offense_total_1st_downs',
    'home_offense_total_3rd_downs',
    'home_offense_total_3rd_down_conversions',
    'home_offense_total_4th_downs',
    'home_offense_total_4th_down_conversions',
    'home_offense_total_seconds_per_play',
    'home_offense_total_1st_downs_per_play',
    'home_offense_total_3rd_down_conversion_pcnt',
    'home_offense_total_4th_down_conversion_pcnt',
    'home_offense_total_punts_per_play',
    'home_offense_total_punts_per_score',
    'home_offense_total_opp_tackles',
    'home_offense_total_opp_solo_tackles',
    'home_offense_total_opp_asst_tackles',
    'offense_total_yards_matchup_differential',
    'offense_total_plays_matchup_differential',
    'offense_total_yards_per_play_matchup_differential',
    'offense_total_1st_downs_matchup_differential',
    'offense_total_3rd_downs_matchup_differential',
    'offense_total_3rd_down_conversions_matchup_differential',
    'offense_total_4th_downs_matchup_differential',
    'offense_total_4th_down_conversions_matchup_differential',
    'offense_total_seconds_per_play_matchup_differential',
    'offense_total_1st_downs_per_play_matchup_differential',
    'offense_total_3rd_down_conversion_pcnt_matchup_differential',
    'offense_total_4th_down_conversion_pcnt_matchup_differential',
    'offense_total_punts_per_play_matchup_differential',
    'offense_total_punts_per_score_matchup_differential',
    'offense_total_opp_tackles_matchup_differential',
    'offense_total_opp_solo_tackles_matchup_differential',
    'offense_total_opp_asst_tackles_matchup_differential',
    'home_penalties_opponent_penalty_yards',
    'home_penalties_opponent_penalty_1st_downs',
    'home_penalties_opponent_yards_per_penalty',
    'penalties_opponent_penalty_yards_matchup_differential',
    'penalties_opponent_penalty_1st_downs_matchup_differential',
    'penalties_opponent_yards_per_penalty_matchup_differential',
    'home_penalties_penalties',
    'home_penalties_yards_per_penalty',
    'penalties_penalties_matchup_differential',
    'penalties_yards_per_penalty_matchup_differential',
    'home_rankings_home_advantage_rating',
    'home_rankings_home_rating',
    'road_rankings_home_advantage_rating',
    'road_rankings_home_rating',
    'rankings_home_rating_matchup_differential',
    'home_rankings_consistency_rating',
    'road_rankings_consistency_rating',
    'rankings_consistency_rating_matchup_differential',
    'home_rankings_luck_rating',
    'road_rankings_luck_rating',
    'rankings_luck_rating_matchup_differential',
    'home_rankings_road_rating',
    'road_rankings_road_rating',
    'home_rankings_sos_basic_rating',
    'road_rankings_sos_basic_rating',
    'rankings_sos_basic_rating_matchup_differential',
    'home_rankings_predictive_rating',
    'road_rankings_predictive_rating',
    'rankings_predictive_rating_matchup_differential',
    'defense_special_teams_fg_made_pcnt_matchup_differential',
    'road_offense_special_teams_gross_punt_ypa',
    'road_offense_special_teams_net_punt_ypa',
    'defense_special_teams_net_punt_ypa_all_matchup_differential',
    'defense_special_teams_gross_punt_ypa_matchup_differential',
    'home_defense_special_teams_gross_punt_yards',
    'home_defense_special_teams_net_punt_yards',
    'home_defense_special_teams_gross_punt_ypa',
    'home_defense_special_teams_net_punt_ypa',
    'road_defense_special_teams_gross_punt_yards',
    'road_defense_special_teams_net_punt_yards',
    'road_defense_special_teams_gross_punt_ypa',
    'road_defense_special_teams_net_punt_ypa',
    'home_defense_passing_attempts',
    'home_defense_passing_completion_pcnt',
    'home_defense_passing_ypa',
    'home_defense_passing_ypc',
    'home_defense_passing_1st_downs',
    'home_defense_passing_td_pcnt',
    'home_defense_passing_pass_rtg',
    'home_defense_passing_sack_pcnt',
    'home_defense_passing_sacks',
    'home_defense_passing_1st_down_pcnt',
    'home_defense_rushing_attempts',
    'home_defense_rushing_ypg',
    'home_defense_rushing_ypa',
    'home_defense_rushing_td_pcnt',
    'home_defense_rushing_1st_down_pcnt',
    'home_defense_scoring_yards_per_point',
    'home_defense_scoring_rz_attempts',
    'home_defense_scoring_rz_td_pcnt',
    'home_def_turnovers_fumbles',
    'home_def_turnovers_takeaways',
    'home_def_turnovers_int_pcnt',
    'home_defense_total_plays',
    'home_defense_total_yards_per_play',
    'home_defense_total_3rd_downs',
    'home_defense_total_4th_downs',
    'home_defense_total_seconds_per_play',
    'home_defense_total_1st_downs_per_play',
    'home_defense_total_3rd_down_conversion_pcnt',
    'home_defense_total_4th_down_conversion_pcnt',
    'home_defense_total_punts_per_play',
    'home_defense_total_punts_per_score',
    'home_defense_total_solo_tackles',
    'home_defense_total_assisted_tackles',
    'road_off_turnovers_fumbles',
    'road_off_turnovers_int_pcnt',
    'def_turnovers_fumbles_matchup_differential',
    'def_turnovers_int_pcnt_matchup_differential',
    'road_offense_passing_attempts',
    'road_offense_passing_incompletions',
    'road_offense_passing_completion_pcnt',
    'road_offense_passing_ypa',
    'road_offense_passing_ypc',
    'road_offense_passing_td_pcnt',
    'road_offense_passing_sack_pcnt',
    'road_offense_passing_1st_down_pcnt',
    'road_offense_passing_pass_rtg',
    'defense_passing_incompletions_matchup_differential',
    'defense_passing_completion_pcnt_matchup_differential',
    'defense_passing_ypa_matchup_differential',
    'defense_passing_ypc_matchup_differential',
    'defense_passing_td_pcnt_matchup_differential',
    'defense_passing_sack_pcnt_matchup_differential',
    'defense_passing_1st_down_pcnt_matchup_differential',
    'defense_passing_pass_rtg_matchup_differential',
    'defense_passing_play_pcnt_matchup_differential',
    'defense_rushing_ypa_matchup_differential',
    'defense_rushing_td_pcnt_matchup_differential',
    'defense_rushing_1st_down_pcnt_matchup_differential',
    'road_offense_scoring_points_per_game',
    'road_offense_scoring_points_per_game_delta',
    'road_offense_scoring_yards_per_point',
    'road_offense_scoring_yards_per_point_delta',
    'road_offense_scoring_points_per_play',
    'road_offense_scoring_rz_td_pcnt',
    'road_offense_total_yards',
    'road_offense_total_plays',
    'road_offense_total_yards_per_play',
    'road_offense_total_1st_downs',
    'road_offense_total_3rd_downs',
    'road_offense_total_3rd_down_conversions',
    'road_offense_total_4th_downs',
    'road_offense_total_4th_down_conversions',
    'road_offense_total_seconds_per_play',
    'road_offense_total_1st_downs_per_play',
    'road_offense_total_3rd_down_conversion_pcnt',
    'road_offense_total_4th_down_conversion_pcnt',
    'road_offense_total_punts_per_play',
    'road_offense_total_punts_per_score',
    'road_offense_total_opp_tackles',
    'road_offense_total_opp_solo_tackles',
    'road_offense_total_opp_asst_tackles',
    'defense_total_yards_matchup_differential',
    'defense_total_plays_matchup_differential',
    'defense_total_yards_per_play_matchup_differential',
    'defense_total_1st_downs_matchup_differential',
    'defense_total_3rd_downs_matchup_differential',
    'defense_total_3rd_down_conversions_matchup_differential',
    'defense_total_4th_downs_matchup_differential',
    'defense_total_4th_down_conversions_matchup_differential',
    'defense_total_time_of_possession_matchup_differential',
    'defense_total_seconds_per_play_matchup_differential',
    'defense_total_1st_downs_per_play_matchup_differential',
    'defense_total_3rd_down_conversion_pcnt_matchup_differential',
    'defense_total_4th_down_conversion_pcnt_matchup_differential',
    'defense_total_punts_per_play_matchup_differential',
    'defense_total_punts_per_score_matchup_differential',
    'defense_total_tackles_matchup_differential',
    'defense_total_solo_tackles_matchup_differential',
    'defense_total_assisted_tackles_matchup_differential',
    'road_penalties_opponent_penalty_yards',
    'road_penalties_opponent_penalty_1st_downs',
    'road_penalties_opponent_yards_per_penalty',
    'road_penalties_penalties',
    'road_penalties_yards_per_penalty',
    'home_days_rest',
    'away_days_rest',
    'rest_differential',
    'week',
    'game_time_hrs',
    'type_id',
    'venue_indoor',
    'home_timezones_traveled',
    'away_timezones_traveled',
    'timezones_traveled_delta',
    'home_travel_distance',
    'away_travel_distance',
    'travel_delta',
]
spread_y = [
    'score_differential',
]
total_y = [
    'total_score', 
]
all_spread_y = [
    '+20.0_home_cover',
    '+19.5_home_cover',
    '+19.0_home_cover',
    '+18.5_home_cover',
    '+18.0_home_cover',
    '+17.5_home_cover',
    '+17.0_home_cover',
    '+16.5_home_cover',
    '+16.0_home_cover',
    '+15.5_home_cover',
    '+15.0_home_cover',
    '+14.5_home_cover',
    '+14.0_home_cover',
    '+13.5_home_cover',
    '+13.0_home_cover',
    '+12.5_home_cover',
    '+12.0_home_cover',
    '+11.5_home_cover',
    '+11.0_home_cover',
    '+10.5_home_cover',
    '+10.0_home_cover',
    '+9.5_home_cover',
    '+9.0_home_cover',
    '+8.5_home_cover',
    '+8.0_home_cover',
    '+7.5_home_cover',
    '+7.0_home_cover',
    '+6.5_home_cover',
    '+6.0_home_cover',
    '+5.5_home_cover',
    '+5.0_home_cover',
    '+4.5_home_cover',
    '+4.0_home_cover',
    '+3.5_home_cover',
    '+3.0_home_cover',
    '+2.5_home_cover',
    '+2.0_home_cover',
    '+1.5_home_cover',
    '+1.0_home_cover',
    '+0.5_home_cover',
    '0.0_home_cover',
    '-0.5_home_cover',
    '-1.0_home_cover',
    '-1.5_home_cover',
    '-2.0_home_cover',
    '-2.5_home_cover',
    '-3.0_home_cover',
    '-3.5_home_cover',
    '-4.0_home_cover',
    '-4.5_home_cover',
    '-5.0_home_cover',
    '-5.5_home_cover',
    '-6.0_home_cover',
    '-6.5_home_cover',
    '-7.0_home_cover',
    '-7.5_home_cover',
    '-8.0_home_cover',
    '-8.5_home_cover',
    '-9.0_home_cover',
    '-9.5_home_cover',
    '-10.0_home_cover',
    '-10.5_home_cover',
    '-11.0_home_cover',
    '-11.5_home_cover',
    '-12.0_home_cover',
    '-12.5_home_cover',
    '-13.0_home_cover',
    '-13.5_home_cover',
    '-14.0_home_cover',
    '-14.5_home_cover',
    '-15.0_home_cover',
    '-15.5_home_cover',
    '-16.0_home_cover',
    '-16.5_home_cover',
    '-17.0_home_cover',
    '-17.5_home_cover',
    '-18.0_home_cover',
    '-18.5_home_cover',
    '-19.0_home_cover',
    '-19.5_home_cover',
    '-20.0_home_cover'
]
all_total_y = [
    '20.0_total_over_hits',
    '20.5_total_over_hits',
    '21.0_total_over_hits',
    '21.5_total_over_hits',
    '22.0_total_over_hits',
    '22.5_total_over_hits',
    '23.0_total_over_hits',
    '23.5_total_over_hits',
    '24.0_total_over_hits',
    '24.5_total_over_hits',
    '25.0_total_over_hits',
    '25.5_total_over_hits',
    '26.0_total_over_hits',
    '26.5_total_over_hits',
    '27.0_total_over_hits',
    '27.5_total_over_hits',
    '28.0_total_over_hits',
    '28.5_total_over_hits',
    '29.0_total_over_hits',
    '29.5_total_over_hits',
    '30.0_total_over_hits',
    '30.5_total_over_hits',
    '31.0_total_over_hits',
    '31.5_total_over_hits',
    '32.0_total_over_hits',
    '32.5_total_over_hits',
    '33.0_total_over_hits',
    '33.5_total_over_hits',
    '34.0_total_over_hits',
    '34.5_total_over_hits',
    '35.0_total_over_hits',
    '35.5_total_over_hits',
    '36.0_total_over_hits',
    '36.5_total_over_hits',
    '37.0_total_over_hits',
    '37.5_total_over_hits',
    '38.0_total_over_hits',
    '38.5_total_over_hits',
    '39.0_total_over_hits',
    '39.5_total_over_hits',
    '40.0_total_over_hits',
    '40.5_total_over_hits',
    '41.0_total_over_hits',
    '41.5_total_over_hits',
    '42.0_total_over_hits',
    '42.5_total_over_hits',
    '43.0_total_over_hits',
    '43.5_total_over_hits',
    '44.0_total_over_hits',
    '44.5_total_over_hits',
    '45.0_total_over_hits',
    '45.5_total_over_hits',
    '46.0_total_over_hits',
    '46.5_total_over_hits',
    '47.0_total_over_hits',
    '47.5_total_over_hits',
    '48.0_total_over_hits',
    '48.5_total_over_hits',
    '49.0_total_over_hits',
    '49.5_total_over_hits',
    '50.0_total_over_hits',
    '50.5_total_over_hits',
    '51.0_total_over_hits',
    '51.5_total_over_hits',
    '52.0_total_over_hits',
    '52.5_total_over_hits',
    '53.0_total_over_hits',
    '53.5_total_over_hits',
    '54.0_total_over_hits',
    '54.5_total_over_hits',
    '55.0_total_over_hits',
    '55.5_total_over_hits',
    '56.0_total_over_hits',
    '56.5_total_over_hits',
    '57.0_total_over_hits',
    '57.5_total_over_hits',
    '58.0_total_over_hits',
    '58.5_total_over_hits',
    '59.0_total_over_hits',
    '59.5_total_over_hits',
]

In [6]:
small_train_df = training_df[train_features + spread_y + total_y + all_spread_y + all_total_y].copy()

In [7]:
cols_with_nulls = small_train_df.columns[small_train_df.isnull().any()]
cols_with_nulls

Index(['offense_special_teams_fg_made_pcnt_matchup_differential',
       'road_defense_passing_td_pcnt', 'road_defense_rushing_td_pcnt',
       'road_defense_total_4th_down_conversion_pcnt',
       'home_offense_passing_td_pcnt', 'home_offense_rushing_td_pcnt',
       'offense_rushing_td_pcnt_matchup_differential',
       'home_offense_scoring_yards_per_point',
       'home_offense_scoring_yards_per_point_delta',
       'home_offense_scoring_rz_td_pcnt',
       'offense_scoring_yards_per_point_matchup_differential',
       'offense_scoring_yards_per_point_delta_matchup_differential',
       'home_offense_total_4th_down_conversion_pcnt',
       'home_offense_total_punts_per_score',
       'offense_total_4th_down_conversion_pcnt_matchup_differential',
       'offense_total_punts_per_score_matchup_differential',
       'home_rankings_home_rating', 'road_rankings_home_rating',
       'rankings_home_rating_matchup_differential',
       'home_rankings_consistency_rating', 'road_rankings_cons

In [8]:
small_train_df.isnull().mean()

game_datetime                                                  0.000000
home_offense_special_teams_net_punt_ypa                        0.000000
offense_special_teams_net_punt_ypa_all_matchup_differential    0.000000
offense_special_teams_gross_punt_ypa_matchup_differential      0.000000
offense_special_teams_fg_made_pcnt_matchup_differential        0.001439
road_defense_passing_attempts                                  0.000000
road_defense_passing_completion_pcnt                           0.000000
road_defense_passing_ypa                                       0.000000
road_defense_passing_ypc                                       0.000000
road_defense_passing_1st_downs                                 0.000000
road_defense_passing_td_pcnt                                   0.000206
road_defense_passing_pass_rtg                                  0.000000
road_defense_passing_sack_pcnt                                 0.000000
road_defense_passing_sacks                                     0

In [9]:
small_train_df["year"] = pd.to_datetime(small_train_df["game_datetime"]).dt.year

In [10]:
X = small_train_df[["year"] + train_features].copy()
y = small_train_df[["year"] + spread_y].copy()

## Try a few models

using rolling windows
average the accuracy and check stdevafter

In [11]:
years = np.arange(2005, 2024)
duration = 8  # Fixed 8-year duration

# Dictionary to store results
results = {
    'model': [],
    'validation_year': [],
    'train_score': [],
    'validation_score': []
}

# Define the models
models = {
    'SVR': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', SVR(kernel='rbf', C=1.0, epsilon=0.2))
    ]),
    'Huber': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', HuberRegressor())
    ]),
    'Random Forest': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor(random_state=54))
    ]),
    'Gradient Boosting': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', GradientBoostingRegressor(random_state=54))
    ]),
    'Extra Trees': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', ExtraTreesRegressor(random_state=54))
    ])
}

# Loop over the years with a fixed 8-year window
for start_year in range(len(years) - duration - 1):  # -1 to leave space for validation year
    train_years = years[start_year:start_year + duration]
    validation_year = years[start_year + duration]
    
    print(f"Train years: {train_years}")
    print(f"Validation year: {validation_year}")
    
    # Filter data for the training and validation periods
    X_train = X[X['year'].isin(train_years)].drop(columns=["year", "game_datetime"])
    y_train = y[y['year'].isin(train_years)].drop(columns=["year"])
    X_test = X[X['year'] == validation_year].drop(columns=["year", "game_datetime"])
    y_test = y[y['year'] == validation_year].drop(columns=["year"])

    # Loop through each model
    for model_name, model in models.items():
        print(f"Training {model_name}")
        
        # Fit the model
        model.fit(X_train, y_train.values.ravel())

        # Predict and evaluate on the training set
        y_train_pred = model.predict(X_train)
        train_score = mean_absolute_error(y_train, y_train_pred)

        # Predict and evaluate on the validation set
        y_pred = model.predict(X_test)
        validation_score = mean_absolute_error(y_test, y_pred)

        # Store results
        results['model'].append(model_name)
        results['validation_year'].append(validation_year)
        results['train_score'].append(train_score)
        results['validation_score'].append(validation_score)
        
        print(f"{model_name} - Train score: {train_score:.4f}")
        print(f"{model_name} - Validation score: {validation_score:.4f}")
        print()

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)


Train years: [2005 2006 2007 2008 2009 2010 2011 2012]
Validation year: 2013
Training SVR
SVR - Train score: 10.5658
SVR - Validation score: 10.4719

Training Huber


/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber - Train score: 10.4049
Huber - Validation score: 11.0845

Training Random Forest
Random Forest - Train score: 4.1808
Random Forest - Validation score: 10.2888

Training Gradient Boosting
Gradient Boosting - Train score: 8.1536
Gradient Boosting - Validation score: 10.5569

Training Extra Trees
Extra Trees - Train score: 0.0000
Extra Trees - Validation score: 9.9006

Train years: [2006 2007 2008 2009 2010 2011 2012 2013]
Validation year: 2014
Training SVR
SVR - Train score: 10.5656
SVR - Validation score: 11.5627

Training Huber


/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber - Train score: 10.4292
Huber - Validation score: 11.6490

Training Random Forest
Random Forest - Train score: 4.1920
Random Forest - Validation score: 11.3726

Training Gradient Boosting
Gradient Boosting - Train score: 8.1584
Gradient Boosting - Validation score: 11.7825

Training Extra Trees
Extra Trees - Train score: 0.0000
Extra Trees - Validation score: 11.2110

Train years: [2007 2008 2009 2010 2011 2012 2013 2014]
Validation year: 2015
Training SVR
SVR - Train score: 10.5685
SVR - Validation score: 10.4513

Training Huber


...

Huber - Train score: 9.6984
Huber - Validation score: 10.4986

Training Random Forest
Random Forest - Train score: 3.9047
Random Forest - Validation score: 10.2012

Training Gradient Boosting
Gradient Boosting - Train score: 7.8509
Gradient Boosting - Validation score: 10.2177

Training Extra Trees
Extra Trees - Train score: 0.0000
Extra Trees - Validation score: 10.2920

Train years: [2013 2014 2015 2016 2017 2018 2019 2020]
Validation year: 2021
Training SVR
SVR - Train score: 9.7904
SVR - Validation score: 11.3932

Training Huber


/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber - Train score: 9.5179
Huber - Validation score: 11.5940

Training Random Forest
Random Forest - Train score: 3.8546
Random Forest - Validation score: 10.8136

Training Gradient Boosting
Gradient Boosting - Train score: 7.7397
Gradient Boosting - Validation score: 10.9391

Training Extra Trees
Extra Trees - Train score: 0.0000
Extra Trees - Validation score: 10.8671

Train years: [2014 2015 2016 2017 2018 2019 2020 2021]
Validation year: 2022
Training SVR
SVR - Train score: 9.9102
SVR - Validation score: 9.3418

Training Huber


/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber - Train score: 9.6469
Huber - Validation score: 10.0263

Training Random Forest
Random Forest - Train score: 3.9304
Random Forest - Validation score: 9.4766

Training Gradient Boosting
Gradient Boosting - Train score: 7.7277
Gradient Boosting - Validation score: 9.6786

Training Extra Trees
Extra Trees - Train score: 0.0000
Extra Trees - Validation score: 9.2544

                model  validation_year  train_score  validation_score
0                 SVR             2013    10.565787         10.471933
1               Huber             2013    10.404913         11.084489
2       Random Forest             2013     4.180847         10.288760
3   Gradient Boosting             2013     8.153560         10.556898
4         Extra Trees             2013     0.000000          9.900640
5                 SVR             2014    10.565647         11.562714
6               Huber             2014    10.429246         11.648994
7       Random Forest             2014     4.192037         11.37259

In [17]:
results_df.groupby(by="model")[["train_score", "validation_score"]].agg(["mean", "std"]).sort_values(by=("validation_score", "mean"))

train_score           validation_score          
                         mean       std             mean       std
model                                                             
Extra Trees          0.000000  0.000000        10.285084  0.697001
Random Forest        4.040795  0.132544        10.331152  0.721249
SVR                 10.203025  0.322861        10.462650  0.792251
Gradient Boosting    8.018861  0.198049        10.561273  0.733804
Huber               10.021156  0.346398        11.074153  0.585577